In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
### Tags: #Source #Partner

### Links: 
___

In [ ]:
# creating a table from s3

query_text = """--sql
    CREATE TABLE db1.uk_addresses_st_partner
    (
        `report_date` Date,
        `partner_uuid_uk`  String,
        `address_uuid` String
    )
    ENGINE = S3('https://storage.yandexcloud.net/dwh-asgard/uk_addresses_st_partner/year=*/month=*/*.csv','CSVWithNames')
    """

ch.query_run(query_text)

In [ ]:
# creating a table for materialized view

query_text = """--sql
    CREATE TABLE db1.uk_addresses_st_partner_ch
    (
       `report_date` Date,
        `partner_uuid_uk` String,
        `address_uuid` String
    )
    ENGINE = MergeTree()
    ORDER BY partner_uuid_uk
    """

ch.query_run(query_text)

In [ ]:
# creating a materialized view

query_text = """--sql
    CREATE MATERIALIZED VIEW db1.uk_addresses_st_partner_mv
    REFRESH EVERY 1 DAY OFFSET 3 HOUR RANDOMIZE FOR 1 HOUR TO db1.uk_addresses_st_partner_ch AS
    SELECT
        *
    FROM db1.uk_addresses_st_partner
    """

ch.query_run(query_text)

___
## Tools
___
### query

In [2]:
query_text = """--sql
    SELECT
        *
    FROM db1.uk_addresses_st_partner_ch
    limit 10
    """

ch.query_run(query_text)

report_date,partner_uuid_uk,address_uuid
date,str,str
2024-10-10,"""0076e1b0-72c3-4783-9306-93d299…","""cb0afec6-4041-4c9b-b9fd-9cd62a…"
2024-10-10,"""0076e1b0-72c3-4783-9306-93d299…","""acc6f2b2-98dc-4657-8f07-0c9b7a…"
2024-10-11,"""0076e1b0-72c3-4783-9306-93d299…","""cb0afec6-4041-4c9b-b9fd-9cd62a…"
2024-10-11,"""0076e1b0-72c3-4783-9306-93d299…","""acc6f2b2-98dc-4657-8f07-0c9b7a…"
2024-10-12,"""0076e1b0-72c3-4783-9306-93d299…","""cb0afec6-4041-4c9b-b9fd-9cd62a…"
2024-10-12,"""0076e1b0-72c3-4783-9306-93d299…","""acc6f2b2-98dc-4657-8f07-0c9b7a…"
2024-10-01,"""0076e1b0-72c3-4783-9306-93d299…","""cb0afec6-4041-4c9b-b9fd-9cd62a…"
2024-10-01,"""0076e1b0-72c3-4783-9306-93d299…","""acc6f2b2-98dc-4657-8f07-0c9b7a…"
2024-10-14,"""0076e1b0-72c3-4783-9306-93d299…","""cb0afec6-4041-4c9b-b9fd-9cd62a…"


### Drop ch

In [ ]:
query_text = """
    DROP TABLE db1.uk_addresses_st_partner_ch
    """
ch.query_run(query_text)

### Drop mv

In [2]:
query_text = """
    DROP TABLE db1.uk_addresses_st_partner_mv
    """
ch.query_run(query_text)

,read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
0,0,0,0,0,0,0,0,1116984,222d40aa-2f40-49bd-b324-52a52560619a
